### Final Exam Part II

Required libraries are imported:

In [301]:
import sqlite3
import pandas as pd
import csv
%matplotlib inline
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from bson import json_util, ObjectId
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize
import numpy as np
import requests
import json, os
import configparser
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import warnings
import random
import pprint
from datetime import datetime
random.seed(datetime.now())
warnings.filterwarnings('ignore')

# Make plots larger
plt.rcParams['figure.figsize'] = (10, 6)

### How to convert SQL database into its NoSQL equivalent?

Data is extracted from tweeter by tweepy in Python and collected data is stored in two excel files named Twitter and Tags. 

Twitter data set contains :id(tweet_id),created_at, favourite_count,retweet_count, user_screen_name	user_followers_count, user_location, urls, media and Hashtags(in untidy form-json) urls, media and text fields.

Tags data set contains :id(tweet_id) and hashtags(tidy form). 

#### SQL 

RSQlite is used.

Data sets are imported to R and twitter data set split up to 2 different tables: User Table and Tweet Table. While Tweet Table contains information directly related with Tweets, USer Table has the information about user such as user_location.
Tags data set is assigned to Hashtag Table. Cleaning, formatting column procedures are applied in R.

In order to satify 1NF to use SQL, primary key is created for Tags data set. Then, those three tables(User Table, Tweet Table, Hashtag Table) are merged together by SQL join function and queries are written,

#### NoSQL 

Since NoSQL is non-relational database and  documents can be created without first defining their structure, Twitter data set is imported to Python and converted to json format again. Python is connected to MongoClient and local host is defined. 

An empty data base is created and named. Afterwards, collection is created in database. Document refers to each record in SQL database. There are few issues I've faced with due to conversion of csv to json. created_at field which includes both time and date information is not considered as Date in Python, instead it is defined as string. In order to solve this problem  to_datetime(datestring) function is used.

In SQL table Hashtags, Tweet and User related queries can be got after joinig those tables with common keys. However, NoSQL json file includes all information by itself. Hashtags are extracted from TEXT column and words having '#' symbols placed in front of  them are found by using tokenizing code indicated below. This code searches the whole string using regular expression.

Stop words are filtered out such that they so not give any additional information for labeling and number of tags used in tweets is found.







In [302]:
#python connected to db file 
sqlite_file = 'project.db'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [303]:
conn.close()

In [304]:
#it is connected to mongodb
import pymongo
conn = pymongo.MongoClient()
conn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [305]:
from pymongo import MongoClient

In [306]:
client = MongoClient()

In [307]:
#local host is specified
MongoClient('localhost', 27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [308]:
db = conn.mydb
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mydb')

In [309]:

db = conn['elif-proj-final-twit']
db

#conn.database_names()

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'elif-proj-final-twit')

In [310]:
#collection is defined
collection = db.twitter
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'elif-proj-final-twit'), 'twitter')

In [311]:
db.collection_names()

['twitter']

In [312]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'elif-proj-final-twit'), 'twitter')

In [313]:
#csv file is imported and converted to json format

def load_csv(csv):
    p=os.path.join("data/", csv)
    print (p)
    data=pd.read_csv(p, encoding = "ISO-8859-1", engine='python')
    return data 
tweets_csv=load_csv('C:/Users/kilic/Desktop/finaldata.csv')
tweets_csv.head()

C:/Users/kilic/Desktop/finaldata.csv


,id,created_at,favourite_count,retweet_count,user_screen_name,user_followers_count,user_location,hashtags,urls,media,text
0,984219499846128000,4/11/2018 23:59,0,24,creativemindgrp,1782,US | CANADA | FRANCE | GERMANY,"[{'text': 'Cannes', 'indices': [21, 28]}]",N,N,RT @Variety: After a #Cannes rule change that ...
1,984219494112428000,4/11/2018 23:59,0,73,paotrrzs,598,Mxico,[],N,N,"RT @westmetromommy: Seriously, if you haven?t ..."
2,984219485954630000,4/11/2018 23:59,0,38,Sense8giflord,484,"Florida, USA",[],N,N,"RT @Global_Cluster: I have somethings to say, ..."
3,984219472675434000,4/11/2018 23:59,0,41,angelicadianna_,267,NaN,[],N,N,RT @baleadalover: I don?t even watch this show...
4,984219470678974000,4/11/2018 23:59,5,6,paotrrzs,598,Mxico,"[{'text': 'RenewEverythingSucks', 'indices': [...",Y,N,@netflix see how much we love it. Please #Rene...


In [314]:
#records are inserted into collection
new =json.loads(tweets_csv.to_json(orient='records'))
new[0]

{'created_at': '4/11/2018 23:59',
 'favourite_count': 0,
 'hashtags': "[{'text': 'Cannes', 'indices': [21, 28]}]",
 'id': 984219499846128000,
 'media': 'N',
 'retweet_count': 24,
 'text': 'RT @Variety: After a #Cannes rule change that bans films without a France theatrical release from playing in competition, @Netflix is sitti?',
 'urls': 'N',
 'user_followers_count': 1782,
 'user_location': 'US | CANADA | FRANCE | GERMANY',
 'user_screen_name': 'creativemindgrp'}

In [315]:
try:
    result = db.twitter.drop()
    print ("analytics tweets dropped")
except:
    pass

analytics tweets dropped


In [316]:
collection.insert(new)

[ObjectId('5adbe4e478c51331e068175d'),
 ObjectId('5adbe4e478c51331e068175e'),
 ObjectId('5adbe4e478c51331e068175f'),
 ObjectId('5adbe4e478c51331e0681760'),
 ObjectId('5adbe4e478c51331e0681761'),
 ObjectId('5adbe4e478c51331e0681762'),
 ObjectId('5adbe4e478c51331e0681763'),
 ObjectId('5adbe4e478c51331e0681764'),
 ObjectId('5adbe4e478c51331e0681765'),
 ObjectId('5adbe4e478c51331e0681766'),
 ObjectId('5adbe4e478c51331e0681767'),
 ObjectId('5adbe4e478c51331e0681768'),
 ObjectId('5adbe4e478c51331e0681769'),
 ObjectId('5adbe4e478c51331e068176a'),
 ObjectId('5adbe4e478c51331e068176b'),
 ObjectId('5adbe4e478c51331e068176c'),
 ObjectId('5adbe4e478c51331e068176d'),
 ObjectId('5adbe4e478c51331e068176e'),
 ObjectId('5adbe4e478c51331e068176f'),
 ObjectId('5adbe4e478c51331e0681770'),
 ObjectId('5adbe4e478c51331e0681771'),
 ObjectId('5adbe4e478c51331e0681772'),
 ObjectId('5adbe4e478c51331e0681773'),
 ObjectId('5adbe4e478c51331e0681774'),
 ObjectId('5adbe4e478c51331e0681775'),
 ObjectId('5adbe4e478c513

In [317]:
for d in collection.find()[:].limit(5):
    print(d)

{'_id': ObjectId('5adbe4e478c51331e068175d'), 'id': 984219499846128000, 'created_at': '4/11/2018 23:59', 'favourite_count': 0, 'retweet_count': 24, 'user_screen_name': 'creativemindgrp', 'user_followers_count': 1782, 'user_location': 'US | CANADA | FRANCE | GERMANY', 'hashtags': "[{'text': 'Cannes', 'indices': [21, 28]}]", 'urls': 'N', 'media': 'N', 'text': 'RT @Variety: After a #Cannes rule change that bans films without a France theatrical release from playing in competition, @Netflix is sitti?'}
{'_id': ObjectId('5adbe4e478c51331e068175e'), 'id': 984219494112428000, 'created_at': '4/11/2018 23:59', 'favourite_count': 0, 'retweet_count': 73, 'user_screen_name': 'paotrrzs', 'user_followers_count': 598, 'user_location': 'M\x82xico', 'hashtags': '[]', 'urls': 'N', 'media': 'N', 'text': 'RT @westmetromommy: Seriously, if you haven?t checked out @EverythingSuxTV on @netflix , you don?t know what you?re missing. It?s not too?'}
{'_id': ObjectId('5adbe4e478c51331e068175f'), 'id': 9842194859

In [318]:
def to_datetime(datestring):    
    dt = datetime.strptime(datestring.strip(), '%Y-%m-%d %H:%M:%S') 
    #dt = dt.utcnow()    
    return dt

In [319]:
import re
def tokenize(txt):  
  txt=re.sub(r'\n', ' ',txt)
  txt=re.compile(r'[\.][ ]+').sub(' ',txt)      
  txt=re.compile(r'[\,][ ]+').sub(' ',txt)    
  txt=re.compile(r'[_+;=!@$%^&\*\"\?]').sub(' ',txt)  
  splitter=re.compile(r'[ ]+')
  # Split the words by non-alpha characters
  words=splitter.split(txt)
  return words
print (tokenize(d['text']))

['', 'netflix', 'see', 'how', 'much', 'we', 'love', 'it', 'Please', '#RenewEverythingSucks']


In [320]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_list = list(stopwords.words('english'))
stop_words={}
for tag in stop_words_list:
    stop_words[tag]=0
print (stop_words.keys())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kilic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
dict_keys(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there

In [321]:
def update_urls_tags(url_list,urls,hashtag_list,hashtags,tag_list,tags):  
  for url in url_list:
      if url in urls:
        urls[url]=urls[url]+1
      else:
        urls[url]=1	 
  for tag in tag_list:
      if tag in tags:
        tags[tag]=tags[tag]+1
      else:
        tags[tag]=1	 
  for hashtag in hashtag_list:
      if hashtag in hashtags:
        hashtags[hashtag]=hashtags[hashtag]+1
      else:
        hashtags[hashtag]=1        
  return urls,hashtags,tags

In [322]:
hashtags={}
starter_tags=['tvshow','tvseries']
for tag in starter_tags:
    hashtags[tag]=0
urls={}
tags={}
print(hashtags)

{'tvshow': 0, 'tvseries': 0}


In [323]:
def extract_tags_urls(dct,words,stop):
  i=0
  tags={} 
  tokens={}     
  urls={}     
  size=len(words)   
  while i < size:
    ngram = words[i]
    i=i+1
    if len(ngram) < 1: continue
    if len(ngram) > 4:        
      if ngram[0:4].lower()=='http':
        if ngram in urls:
          urls[ngram]=urls[ngram]+1
        else:
          urls[ngram]=1	 
    if ngram[0]=='#':
  #    ngram=re.sub(r'\#', '',ngram)     if you want to remove the # 
      tags[ngram]=1 
    if ngram.lower() not in stop:    
        tokens[ngram]=1	          
    if ngram in dct:
      tags[ngram]=1
    if i < (size-1):
      ngram = words[i] + ' ' + words[i+1]
      if words[i].lower() not in stop:    
        tokens[ngram]=1	        
      if ngram in dct:
        tags[ngram]=1
    if i < (size-2):
      ngram = words[i] + ' ' + words[i+1] + ' ' + words[i+2]           
      if ngram in dct:
        tags[ngram]=1
  return list(tags.keys()),list(urls.keys()),list(tokens.keys())
print (extract_tags_urls(hashtags,(tokenize(d['text'])),stop_words))

(['#RenewEverythingSucks'], [], ['netflix', 'see how', 'see', 'much we', 'much', 'love it', 'love', 'Please #RenewEverythingSucks', 'Please', '#RenewEverythingSucks'])


In [324]:
cnt=0
for tweet in new:
   #
    retweet_count=0
    try:
        retweet_count=int(tweet['retweet_count'])
    except:        
        pass   
    tweet_tags,tweet_urls,tweet_ngrams=extract_tags_urls(hashtags,(tokenize(tweet['text'])),stop_words)
    print (tweet_tags)
    urls,hashtags,tags=update_urls_tags(tweet_urls,urls,tweet_tags,hashtags,tweet_ngrams,tags)
    try:
        
        #j=tweet_json(tweet['id'],tweet['text'],ts,favorite_count,retweet_count,tweet_urls,tweet_tags)
        j=new(tweet['Date'], tweet['Time'], tweet['id'],tweet['text'],tweet['user_screen_name'], tweet['user_location'], tweet['user_followers_count'],retweet_count,tweet_tags)
        result = collection.insert_one(j)
        cnt+=1
    except:
        pass       
print ("%d tweets inserted."%cnt)


['#Cannes']
[]
[]
[]
['#RenewEverythingSucks']
[]
[]
[]
[]
['#RenewEverythingSucks']
[]
[]
[]
[]
['#ToAllTheBoysIveLovedBefore']
[]
[]
[]
[]
['#reneweverythingsucks']
[]
[]
[]
['#RenewEverythingSucks']
['#iamkatemessner']
[]
['#RenewGypsy', '#GypsyMatters']
['#reneweverythingsucks', '#iamkatemessner']
[]
['#WhenYou']
[]
['#RenewGypsy', '#GypsyMatters']
[]
[]
[]
['#RenewEverythingSucks']
['#Banff2018']
['#RenewEverythingSucks']
['#reneweverythingsucks']
[]
[]
['#RenewEverythingSu']
[]
[]
[]
[]
['#reneweverythingsucks']
['#RenewEverythingSucks']
['#Philippines', '#Duterte']
[]
['#reneweverythingsucks']
[]
['#StreamTeam', '#me', '#reviews']
[]
[]
[]
['#HBTL']
[]
['#HBTL']
[]
[]
[]
[]
['#reneweverythingsucks']
['#RenewEverythingSucks']
['#RenewEverythingSucks', '#SaveEverythingSucks']
[]
[]
[]
['#Yodeldale']
[]
['#reneweverythingsucks']
[]
[]
[]
[]
['#ThingsImAfraidOfLosing']
[]
['#RenewEverythingSucks']
[]
[]
['#RenewOnMyBlock']
[]
[]
['#Movies', '#RoxanneRoxanne', '#MovieReview', '#Dope'

['#RenewEverythingSucks']
[]
['#RenewEverythingSucks']
[]
[]
['#RenewEverthingSucks']
['#RenewEvery']
[]
[]
['#RenewEverythingSucks']
[]
['#RenewEverythingSucks']
[]
[]
[]
[]
[]
['#reneweverythingsucks']
['#RenewEverythingSucks']
[]
['#reneweverythingsucks', '#iamemalineaddario', '#iamkatemessner', '#weareboringhigh', '#representationmatters']
['#RenewEverythingSucks']
['#RenewEverythingSucks']
['#RenewEverythingSucks']
[]
[]
[]
['#RenewEverythingSucks']
[]
[]
[]
[]
['#RenewEverythingSucks']
['#renewEverythingSucks']
['#reneweverythingsucks', '#netflix']
['#RenewEverythingSucks']
[]
[]
[]
['#RenewEverythingSucks']
[]
[]
['#uglydelicious']
['#RenewEverythingSucks']
['#renewonmyblock']
['#NailedIt']
['#Gypsy', '#Gypsy2']
['#RenewEverythingSucks']
['#Safe']
[]
[]
['#RenewEverthingSucks']
[]
[]
['#RenewEverythingSucks']
['#RenewEverythingSucks']
['#RenewEverythingSucks']
[]
['#reneweverythingsucks']
[]
['#PrettyLittleLiars']
['#RenewEvery']
['#RenewEverythingSucks']
[]
[]
['#RenewEverthing

### i. What are people saying about netflix?

IN SQL database, Hashtag Table is used to answer this question. Since each record/row contains single entity(hashtag), simple count query in SQl can be used. However, in NoSQL there is not Hashtag Table and hashtags have to extracted from Text field of documents leading much more complex counting function to be written. Count function, searches hashtags in each text field of document and continues searching left documents. This function keep track of hashtag itself and the number of times it is used by (key, value) structure. Key refers to hashtag and value is the number which is updated in hashtag search process.

In [325]:
for key, value in hashtags.items():
    print ("%s count %d"%(key, value))

tvshow count 0
tvseries count 0
#Cannes count 3
#RenewEverythingSucks count 302
#ToAllTheBoysIveLovedBefore count 2
#reneweverythingsucks count 148
#iamkatemessner count 24
#RenewGypsy count 24
#GypsyMatters count 20
#WhenYou count 1
#Banff2018 count 1
#RenewEverythingSu count 2
#Philippines count 1
#Duterte count 1
#StreamTeam count 1
#me count 1
#reviews count 1
#HBTL count 2
#SaveEverythingSucks count 9
#Yodeldale count 3
#ThingsImAfraidOfLosing count 1
#RenewOnMyBlock count 7
#Movies count 1
#RoxanneRoxanne count 1
#MovieReview count 1
#Dope count 1
#HipHop count 1
#Blerd count 1
#Nerd count 1
#Geek count 1
#JayZ count 1
#SaveFore count 1
#NetflixWeNeedSupernatural count 1
#eusoupercystente count 2
#fastestcar count 1
#TeamSalamone count 1
#winning count 1
#SaveFor count 1
#RandomForeverThought count 1
#SaveForever count 1
#nosen count 1
#Diablero count 2
#Onmyblock count 2
#Travelers count 2
#EverythingSucks count 12
#Netflix count 14
#LostInSpace count 16
#RenewEverthingSucks cou

In order to answer this question, scope is limited with used hashtags. If people is talking about something, it is assumed that he uses related hashtags.

For the last 1 month, most of the tweets mentioning @netflix is related with tv show named "EverythingSucks". RenewEverythingSucks, reneweverythingsucks and SaveEverythingSucks hashtags are used more than 459 times in total indicating that people want this show to continue.

LostinSpace(16) and JessicaJos(11) are the other popular tv shows. RenewGypsy is counted 24 times and considering another tag related with Gypsy - GypsyMatters(20), people also have talked about Gypsy and want tv series to continue.

Comedy, Cartoons, News are the hashtags indicating genre while fail, me, FridayNight, goodfridaynight are defining mood of the user. Although, genre, mood, actor name etc. are used as hashtags, we can definitely say that people frequently talk about tv shows directly as it is expected.



### ii. How viral are my posts?

Virality of a post can be measured in terms of total of retweets it gets. Since, retweets are the primary way to spread tweets. 

So, I defined Virality V = Total Retweets

This formula can be used to find either virality of a single post or virality of all posts of user in a certain time period.

In this case, Virality of users named "Libbybyerley1" and "SanversArgent" is found and their posts's virality is compared. Following code returns id of their tweets and number of rt's they get respectively.

In total Libbybyerley1's 3 tweets have 43 retweets in total. Thus, V =43 (Virality of posts' between 4/11/2018 - 4/13/2018)

In total SanversArgent's 3 tweet have 35 retweets. V= 35 (Virality of posts between 4/11/2018 - 4/13/2018)

As a result, Libbybyerley1's total virality is greater than SanversArgent's total virality. However; in terms of average measurement, SanversArgent has more virality.

In [326]:
results1 = collection.find({'user_screen_name': "Libbybyerley1"},{"retweet_count":1,"id":1})
for d in results1:
    print(d)

{'_id': ObjectId('5adbe4e478c51331e0681777'), 'id': 984219153908290000, 'retweet_count': 13}
{'_id': ObjectId('5adbe4e478c51331e068177c'), 'id': 984219115387834000, 'retweet_count': 13}
{'_id': ObjectId('5adbe4e478c51331e0681830'), 'id': 984215972742615000, 'retweet_count': 17}


In [327]:
results2 = collection.find({'user_screen_name': "SanversArgent"},{"retweet_count":1,"id":1})
for d in results2:
    print(d)


{'_id': ObjectId('5adbe4e478c51331e068179e'), 'id': 984218521046577000, 'retweet_count': 35}


### iii.  How much influence to my posts have?

In SQL databases used query is somehow similar to mongodb code. In SQL TweetTable is used to get desired information while in NoSQL FinalData set is used.

Influence of a post can be measured in terms of total number of likes and retweets he gets with his tweets. Since, retweets and likes demonstrate how much a post gets people's attention. People's likes and retweets are seen in timeline of tweeter resulting in more people to notice the tweet. Therefore, retweets and likes are chosen as indicators of influence.

So simply, Influence I = SUM(likes) + SUM(retweets)
This formula can be used to find either influence of a single post or influence of all posts of user in a certain time period.

In this case, influence of users named "paotrrzs" and "PolandCluster" is found and their posts's influence is compared. Following code returns id of their tweets and mumber of likes and fav they get respectively.

In total paotrrzs's tweets have 26 likes and 172 retweets. I= 26 + 172 =178 (Influence of posts' between 4/11/2018 - 4/13/2018)

In total PolandCluster's tweets have 3 likes and 93 retweets. I= 3 + 93 = 96 (Influence of posts' between 4/11/2018 - 4/13/2018)

As a result, paotrrzs's posts have more influence than PolandCluster's posts.

In [328]:
results3 = collection.find({'user_screen_name': "paotrrzs"},{"retweet_count":1, "favourite_count":1,"id":1})
for d in results1:
    print(d)

In [329]:
results4 = collection.find({'user_screen_name': "PolandCluster"},{"retweet_count":1, "favourite_count":1,"id":1})
for d in results2:
    print(d)

### xi. Should I add and picture or url to my post?

If tweet includes any media or url this information is stored in tweets and is extracted when tweets are collected by API. If tweet does not include media or url fiels of those attributes are seen as blank. WHen I stored collected data in csv, I replace those information with 'Y' and 'N'. 

Y in urls or media field refers to tweet including URL/Media.
N in urls or media field refers to tweet without any URL or Media.

Csv file which has 'Y' or 'N' in media/urls field is used to create NoSQL database.

In order to answer this question, number of retweets and likes are considered, If tweet having media or url gets many likes or retweets, I should add them to my post to increase my tweets interaction. Average number of likes and Averafe number of retweets of tweets with URL/Media are compared with tweets without them.

In [330]:
results5= collection.aggregate([{'$group': {'_id':"$media", 'avgFav': {'$avg':"$favourite_count"} } }])
for d in results5:
    print(d)

{'_id': 'Y', 'avgFav': 3.3244444444444445}
{'_id': 'N', 'avgFav': 1.7456375838926175}


In [331]:
results6= collection.aggregate([{'$group': {'_id':"$media", 'avgRT': {'$avg':"$retweet_count"} } }])
for d in results6:
    print(d)

{'_id': 'Y', 'avgRT': 1030.4666666666667}
{'_id': 'N', 'avgRT': 68.99194630872483}


Results indicate that tweets with media get more retweets and likes compared with tweets without media. Especially, avgRT that tweets with media get is 15 times higher than tweets without media get.

In [332]:
results7= collection.aggregate([{'$group': {'_id':"$urls", 'avgFav': {'$avg':"$favourite_count"} } }])
for d in results7:
    print(d)

{'_id': 'Y', 'avgFav': 5.307888040712468}
{'_id': 'N', 'avgFav': 0.9553706505295008}


In [333]:
results8= collection.aggregate([{'$group': {'_id':"$urls", 'avgRT': {'$avg':"$retweet_count"} } }])
for d in results8:
    print(d)

{'_id': 'Y', 'avgRT': 115.82442748091603}
{'_id': 'N', 'avgRT': 218.70953101361573}


Results for tweets including urls is slighlty differerent than what we observed in media case. Although, similar results are observed for average likes (avgFav is greater for tweets having urls), average number of retweets is higher for tweets without urls.

Therefore, I should definitely add media to increase my interaction, likes, rt's. Using urls also increases my likes, so I can also add them to my post.